In [2]:
import pandas as pd

# Make Splits and processing

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
l1='en'
l2='es'

In [6]:
import pandas as pd

In [7]:
df=pd.read_csv(f'data/{l1}_{l2}_dataset.csv')

In [8]:
df.head()

,ID,en,en_meaning,es,es_meaning,type,lexical_category
0,1,nature,the external world in its entirety,naturaleza,The set of everything that exists and that is ...,COGNATE,N
1,2,culture,"the customary beliefs, social forms, and mater...",cultura,"Set of ways of life and customs , knowledge an...",COGNATE,N
2,3,restaurant,a business establishment where meals or refres...,restaurante,Public establishment where food and drinks are...,COGNATE,N
3,4,traffic,"the vehicles, pedestrians, ships, or planes mo...",tráfico,"Movement or transit of people , goods , etc., ...",COGNATE,N
4,5,tourist,one that makes a tour for pleasure or culture,turista,"To trade , to negotiate with money and goods.",COGNATE,N


In [9]:
def remove_space(x):
    x=x.replace('\n','')
    x=x.strip('.')
    x=x.strip()
    x=x+'.'
    x=x.lower()
    x=x.replace(' , ',', ')
    x=x.replace(" '","'")
    return x.capitalize()

In [10]:
df[f'{l2}_meaning']=df[f'{l2}_meaning'].apply(remove_space)
df[f'{l1}_meaning']=df[f'{l1}_meaning'].apply(remove_space)

In [11]:
from sentence_transformers import SentenceTransformer
SENTENCE_TRANSFORMER_PATH='/home/models/all-MiniLM-L6-v2'
model = SentenceTransformer(SENTENCE_TRANSFORMER_PATH)

/home/eshaan/multilingual-cognates/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [12]:
l1_meaning=df[f'{l1}_meaning'].to_list()
l2_meaning=df[f'{l2}_meaning'].to_list()
word_type=df['type'].to_list()

In [13]:
len(word_type)

420

In [14]:
def find_negative_samples(meaning):
    embeddings=model.encode(meaning)
    similarities=model.similarity(embeddings,embeddings)
    negative_sample_index=[]
    for sim in similarities:
        similarity=sim.clone()
        index_samples=(similarity>=0.6).nonzero().squeeze()
        similarity[index_samples]=-float('inf')
        negative_samples=similarity.argmax().item()
        negative_sample_index.append(negative_samples)
    return [meaning[i] for i in negative_sample_index]

In [15]:
l1_neg_meaning=find_negative_samples(l1_meaning)
l2_neg_meaning=find_negative_samples(l2_meaning)

In [16]:
def get_negative_samples(neg_sampll2_meaning,l2_meaning,word_type):
    neg=[]
    for i,wt in enumerate(word_type):
        if wt=='FALSE COGNATE':
            neg.append(l2_meaning[i])
        else:
            neg.append(neg_sampll2_meaning[i])
    return neg

In [17]:
df[f'{l1}_neg_meaning']=get_negative_samples(l1_neg_meaning,l2_meaning,word_type)
df[f'{l2}_neg_meaning']=get_negative_samples(l2_neg_meaning,l1_meaning,word_type)

In [18]:
df.to_csv(f'data/{l1}_{l2}_processed.csv')

In [19]:
df=pd.read_csv(f'data/{l1}_{l2}_processed.csv')

In [20]:
test_df,train_df=train_test_split(df,test_size=0.05,random_state=1,stratify=df['type'])

In [21]:
len(test_df)

399

In [22]:
len(train_df)

21

In [23]:
test_df.columns

Index(['Unnamed: 0', 'ID', 'en', 'en_meaning', 'es', 'es_meaning', 'type',
       'lexical_category', 'en_neg_meaning', 'es_neg_meaning'],
      dtype='object')

In [24]:
test_df['type'].value_counts()

type
COGNATE          133
FALSE COGNATE    133
NON COGNATE      133
Name: count, dtype: int64

In [25]:
train_path=f'data/{l1}_{l2}_train.csv'
test_path=f'data/{l1}_{l2}_test.csv'

In [26]:
train_df.to_csv(train_path)
test_df.to_csv(test_path)

In [27]:
len(test_df)

399

In [28]:
len(train_df)

21